In [11]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk 
import string
import pdb
import utils
import bayes_test

from nltk.corpus import twitter_samples
from nltk.tokenize import TweetTokenizer
from os import getcwd

In [12]:
all_positive_tweets=twitter_samples.strings("positive_tweets.json")
all_negative_tweets=twitter_samples.strings("negative_tweets.json")
train_pos=all_positive_tweets[:4000]
test_pos=all_negative_tweets[4000:]
train_neg=all_negative_tweets[:4000]
test_neg=all_negative_tweets[4000:]
train_x=train_pos+train_neg
test_x=test_pos+test_neg
#label
train_y=np.append(np.ones(len(train_pos)),np.zeros(len(train_neg))) 
test_y=np.append(np.ones(len(test_pos)),np.zeros(len(test_neg))) 

##### preprocessing

In [13]:
from utils import *
print(train_x[6])
process_tweet(train_x[6])

We don't like to keep our lovely customers waiting for long! We hope you enjoy! Happy Friday! - LWWF :) https://t.co/smyYriipxI


['like',
 'keep',
 'love',
 'custom',
 'wait',
 'long',
 'hope',
 'enjoy',
 'happi',
 'friday',
 'lwwf',
 ':)']

In [27]:
def count_tweets(result,tweets,ys):
    for y,tweet in zip(ys,tweets):
        for word in process_tweet(tweet):
            pair=(word,y)
            if pair in result:
                result[pair]+=1
            else:
                result[pair]=1
    return result

In [28]:
bayes_test.test_count_tweets(count_tweets)

 All tests passed


##### create freq

In [29]:
freqs=count_tweets({},train_x,train_y)
len(freqs)

11427

##### Train

In [30]:
def train_bayes(freqs,train_x,train_y):
    loglikelihood={}
    logprior=0
    
    vocab={word:0 for word,sentiment in freqs.keys()}
    V=len(vocab)

    N_pos=0
    N_neg=0
    for pair in freqs.keys():
        if pair[1]>0:
            N_pos+=freqs[pair]
        else:
            N_neg+=freqs[pair]

    D=len(train_x)
    D_pos=np.sum(train_y==1)
    D_neg=np.sum(train_y==0)
    logprior=np.log(D_pos/D_neg)

    for word in vocab.keys():
        freq_pos=freqs.get((word,1),0)
        freq_neg=freqs.get((word,0),0)

        p_w_pos = (freq_pos+1)/(N_pos + V)
        p_w_neg = (freq_neg+1)/(N_neg + V)

        loglikelihood[word] = np.log(p_w_pos/p_w_neg)
    return logprior, loglikelihood

In [31]:
logprior, loglikelihood = train_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

0.0
9161


##### test

In [33]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    word_l = process_tweet(tweet)
    p = 0
    p += logprior
    for word in word_l:
        if word in loglikelihood:
            p += loglikelihood[word]
    return p

In [35]:
bayes_test.test_naive_bayes_predict(naive_bayes_predict)

 All tests passed


In [36]:
my_tweet = 'She smiled.'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print('The expected output is', p)

The expected output is 1.557492820301094


In [ ]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood, naive_bayes_predict=naive_bayes_predict):
    accuracy = 0
    y_hats = []
    for tweet in test_x:
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            y_hat_i = 1
        else:
            y_hat_i = 0
        y_hats.append(y_hat_i)
    error = np.sum(abs(y_hats-test_y))/len(y_hats)
    accuracy = 1-error
    return accuracy